# Harnessing Matplotlib & Astropy

## Matplotlib.pyplot

`matplotlib.pyplot` is a powerful graphics module often used by Python developers. This is by no means a comprehensive tutorial, but it will point out a few important things about `matplotlib.pyplot` objects and give advice about incorporating plotting functionality in your open-source astronomy software.

There are other (excellent) plotting tools in common practice in astronomy, especially plotly and bokeh. We encourage you to look into these tools as well, but we'll only discuss `matplotlib.pyplot` here.

The basic "unit" of `matplotlib.pyplot` is [the `Figure` object](https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.figure.Figure.html#matplotlib.figure.Figure), which acts as container for all the other plot elements. Let's look at a few different ways to create a `Figure` object:




In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# create some sample data
x = np.array([1, 2, 3, 4])
y = np.array([1, 4, 9, 16])
y_error = np.array([5, 1.2, 2, 5])

fig = plt.figure() # initializes a `Figure` object
plt.errorbar(      # plots onto the current `Figure`
    x, y, y_error, ls='', fmt="*"  
)  

print(type(fig))

The current figure changes when you initialize a new `Figure` object. Unless you specify a particular `Figure` object, `plt.___()` functions will always add to the current `Figure` object. That is a nice feature-- it makes it easy to add multiple types of plots onto the same figure.

Let's look at another way to initialize a new `Figure` object:

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

ax[0].plot(x, y)
ax[1].plot(x, y)
ax[1].set_yscale('log')

print(type(fig))
print(type(ax))
print(type(ax[0]))

Here we've encounter another kind of object: [the `Axes` object](https://matplotlib.org/api/axes_api.html#matplotlib.axes.Axes). These objects are containers for all the plot elements *in a particular figure panel*. `Axes` are attributes of `Figure` objects:

In [ ]:
fig.axes[0].set_xlim(1, 3)
fig

Two useful methods to keep in mind are `plt.gca()` and `plt.sca()` (get/set current `Axes`). Let's see how they work:

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2)

mystery_axes = plt.gca()              # get current Axes
mystery_axes.scatter(x, y, color='k') # acts on `mystery_axes`

plt.sca(ax[0,0])        # set current Axes
plt.plot(x, y, ls='-.') # acts on current Axes
plt.tight_layout()      # prettify

#### Activity #1 

Let's practice working with `Figure` and `Axes` objects. Run the code snippet below to generate some orbits and plot them using `orbitize!`

In [ ]:
import orbitize.driver

# set up fit
myDriver = orbitize.driver.Driver(
    '{}/GJ504.csv'.format(orbitize.DATADIR),
    'OFTI', 1, 1.22, 56.95, mass_err=0.08, plx_err=0.26
)
s = myDriver.sampler

# run orbitize!
orbits = s.run_sampler(100)
myResults = s.results

# make plot
epochs = myDriver.system.data_table['epoch']
orbit_figure = myResults.plot_orbits(
    start_mjd=epochs[0]
)

Notice that the last line of this snippet returns the `Figure` object used to make the plot above. This is good practice-- it lets your users more easily customize figures, just as you're about to do!

Tasks:
- Change the x-axis limits of the top right panel to 2011-2015.
- Add a black star at (0, 0) in the leftmost panel.
- Change the dpi (dots per inch, a measure of resolution) of the Figure to 250.
- Save the figure to your Desktop using [`plt.savefig()`](https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.savefig.html). 

### Activity #2

Use gridspec (Google it!) to create a basic [corner plot](https://corner.readthedocs.io/en/latest/pages/quickstart.html). Plot the samples generated in the code snippet below. Your corner plot must:
- display histograms along the diagonal
- display a contour plot along the non-diagonal
- handle 2-dimensional arrays

Your corner plot may, if you have time/interest:
- be super pretty
- handle N-dimensional arrays

In [ ]:
import numpy as np

ndim, nsamples = 2, 10000
samples = np.random.randn(ndim * nsamples)

print(samples)

# Astropy

astropy is full of good stuff. Let's do some examples with units, constants, io, tables, time, coordinates, fits files

googling a mysterious error

don't reinvent the wheel!
check out more tutorials: http://learn.astropy.org/tutorials.html